In [1]:
import numpy as np
import pandas as pd
import os
from google_drive_downloader import GoogleDriveDownloader as gdd
# pip install googledrivedownloader
# Download data from Drive
download = False

In [2]:
def _download_data(download=True):
    if download:
        gdd.download_file_from_google_drive(
            file_id="1-e8PVHbFu_LaMKConDKeL7GPJdl4yZQB",
            dest_path="./data_temp/Microdato_Censo2017-Viviendas.zip",
            unzip=True
        )
    return True

In [3]:
_download_data(download)
df = pd.read_csv("./data_temp/Microdato_Censo2017-Viviendas.csv", sep=";", low_memory=False)

In [4]:
questionnaire = pd.ExcelFile("https://docs.google.com/spreadsheets/d/e/2PACX-1vQ4xZxDpyDY4NursNbrsWlCqjREdmBbfC1EMlz4UGQe7M8wMA7Mqw8tZbAcBdkjgBzloyQdcnhiv10C/pub?output=xlsx")
labels = pd.read_excel(questionnaire, "Labels")

In [5]:
_drop = ["REGION", "PROVINCIA", "ZC_LOC", "ID_ZONA_LOC", "NVIV"]
_groupby = ["COMUNA", "AREA", "DC", "P01", "P02", "P03A", "P03B", "P03C", "P04", "P05"]

q = df.drop(columns=_drop).groupby(_groupby)
_ = pd.DataFrame({
    "cant_viv" : q.size(),
    "cant_hog" : q["CANT_HOG"].agg('sum'),
    "cant_per" : q["CANT_PER"].agg('sum')
}).reset_index()

In [6]:
_ = _.rename(columns = { 
    key: labels[labels["id"] == key].iloc[0]["label"] for key in _groupby
})

In [7]:
geo_datachile_id = pd.read_csv("https://raw.githubusercontent.com/datachile/datachile-etl/master/official_ids/2017_06_27_comunas_datachile_fixed.csv")
geo_datachile_id = geo_datachile_id.drop(columns=["region_id", "region_name", "comuna_tax_office_id"])

geo_datachile_id = geo_datachile_id.fillna(0).astype({"comuna_customs_id": "int"})
geo_datachile_id = geo_datachile_id.rename(columns={"comuna_customs_id": "comuna_id"})

In [8]:
_ = pd.merge(_, geo_datachile_id, on="comuna_id", how="inner")

In [9]:
_ = _.drop(columns=["comuna_id", "comuna_name"])

In [10]:
pd.DataFrame(_).to_csv("data_final/housing_census.csv", index=False)